In [1]:
import glob
import os
import pandas as pd
import numpy as np

In [2]:
#specify folder with data
datafolder = r"C:///*.asc"

#define experiment name or name of the folder is experiment name
experiment_name = r""

#
from preprocessing import get_folder_name
if experiment_name: 
    pass 
else:
    experiment_name = get_folder_name(datafolder)

In [3]:
#choose between method of catching datafiles
#datafiles = glob.glob(os.path.join(datafolder, '*'))
datafiles = glob.glob(datafolder)

#excluding the *_average.asc-files
filtered_files = [f for f in datafiles 
                  if "averaged" not in os.path.basename(f).lower()]

In [4]:
#base-data extraction (angle, temperature, wavelength, refractive_index, viscosity)
from preprocessing import extract_data

all_data = []
for file in filtered_files:
  extracted_data = extract_data(file)
  if extracted_data is not None:
     filename = os.path.basename(file)
     extracted_data['filename'] = filename  
     all_data.append(extracted_data)
      
if all_data:
  df_basedata = pd.concat(all_data, ignore_index=True)
  df_basedata.index = df_basedata.index + 1  
else:
  print("No data extracted!")

In [5]:
#calculate q and q^2 from basedata
df_basedata['q'] = abs(((4*np.pi*df_basedata['refractive_index'])/(df_basedata['wavelength [nm]']))*np.sin(np.radians(df_basedata['angle [°]'])/2))
df_basedata['q^2'] = (df_basedata['q']**2)

In [6]:
#check basedata
run_cell_basedata = False

#
if run_cell_basedata:
    print(df_basedata)
else:
    print("Basedata check skipped.")

Basedata check skipped.


In [7]:
#extract scattering intensity (not required for DLS)
perform_intensity_processing = False
from intensity import extract_intensity

#
if perform_intensity_processing:
    intensity_data = []
    for file in filtered_files:
      extracted_intensity = extract_intensity(file)
      if extracted_intensity is not None:
         filename = os.path.basename(file)
         extracted_intensity['filename'] = filename  
         intensity_data.append(extracted_intensity)
      
    if intensity_data:
      df_intensity = pd.concat(intensity_data, ignore_index=True)
      df_intensity.index = df_intensity.index + 1  
    else:
      print("No data extracted!")
else:
    print("Intensity processing skipped.")

Intensity processing skipped.


In [8]:
#process and plot scattering intensity (not required for DLS)
if perform_intensity_processing:
    from intensity import plot_meancr
    df_intensity['MeanCR_corr [kHz]'] = ((df_intensity['meancr0 [kHz]'] + df_intensity['meancr1 [kHz]'])/2/(df_intensity['monitordiode [cps]']*10**(-3))*np.sin(np.radians(df_intensity['angle [°]'])))
    plot_meancr(df_intensity, 'angle [°]', 'MeanCR_corr [kHz]')
else:
    pass

In [9]:
#extract countrates
from preprocessing import find_countrate_row, extract_countrate
all_countrates = {}  #dictionary to store the Countrate-dataframes
for file in filtered_files:
    extracted_countrate = extract_countrate(file)
    if extracted_countrate is not None:
        filename = os.path.basename(file)
        all_countrates[filename] = extracted_countrate

#rename columns accordingly
new_column_names_for_countrate = {0: 'time [s]', 1: 'detectorslot 1', 2: 'detectorslot 2', 3: 'detectorslot 3', 4: 'detectorslot 4'}
all_countrates = {key: df.rename(columns=new_column_names_for_countrate) for key, df in all_countrates.items()}

In [10]:
#check countrate-dataframes
run_cell_countrates = False

#
if run_cell_countrates:
    if all_countrates:  
        for filename, df in all_countrates.items():
            print(f"\nCountrate for {filename}:")
            print(df)
    else:
        print("No countrate extracted!") 
else:
    print("Countrate check skipped.")

Countrate check skipped.


In [11]:
#plotting all countrates
plot_countrate_graphs = False
allow_data_filtering = True 

#
from preprocessing import plot_countrates, cli_countrate_exclusion
if plot_countrate_graphs:  
    if allow_data_filtering:
        filtered_countrates = cli_countrate_exclusion(all_countrates)
        if filtered_countrates:
            original_countrates = all_countrates
            all_countrates = filtered_countrates
            print("Analysis will continue with the filtered dataset")
    else: #just plot without filtering
        plot_countrates(all_countrates, show_indices=False)
else:
    print("Plot countrates skipped.")

Plot countrates skipped.


In [12]:
#extract correlation based on filtered countrate data
all_correlations = {}
#get the original file paths dictionary (assuming it was created earlier)
#this maps from filename to full file path
file_to_path = {os.path.basename(file): file for file in filtered_files}

from preprocessing import find_correlation_row, extract_correlation

#process only the files that remain after countrate filtering
for filename in all_countrates.keys():
    if filename in file_to_path:
        file_path = file_to_path[filename]
        extracted_correlation = extract_correlation(file_path)
        if extracted_correlation is not None:
            all_correlations[filename] = extracted_correlation
    else:
        print(f"Warning: No matching file found for {filename} in filtered_files")

print(f"Extracted correlation data for {len(all_correlations)} files")

#rename columns accordingly
#  !!! depending on the measurement-settings, some columns may be zero !!!
new_column_names_for_correlations = {0: 'time [ms]', 1: 'correlation 1', 2: 'correlation 2', 3: 'correlation 3', 4: 'correlation 4'}
all_correlations = {key: df.rename(columns=new_column_names_for_correlations) for key, df in all_correlations.items()}

Extracted correlation data for 39 files


In [13]:
#check correlation-dataframes
run_cell_correlations = False

#
if run_cell_correlations:
    if all_correlations:  
        for filename, df in all_correlations.items():
            print(f"\nCorrelation for {filename}:")
            print(df)
    else:
        print("No correlation extracted!") 
else:
    print("Correlation check skipped.")

Correlation check skipped.


In [14]:
#plotting all normalized correlations (g(2)-1)
plot_correlation_graphs = False
allow_correlation_filtering = True

#
from preprocessing import plot_correlations, cli_correlation_exclusion
if plot_correlation_graphs:
    if allow_correlation_filtering:
        filtered_correlations = cli_correlation_exclusion(all_correlations)
        if filtered_correlations:
            original_correlations = all_correlations
            all_correlations = filtered_correlations
            print("Correlation analysis will continue with the filtered dataset")
    else: #just plot without filtering
        plot_correlations(all_correlations, show_indices=False)
else:
    print("Plot correlation skipped.")

Plot correlation skipped.


In [15]:
#possibility to remove certain data for evaluation
from preprocessing import remove_from_data, remove_dataframes
#check if filtered data already exists from previous steps
if 'filtered_correlations' in locals() and filtered_correlations != all_correlations:
    print("Using already filtered correlation data from the exclusion process.")
    all_correlations_mod = filtered_correlations
else:
    # !!! if not explicitly filtered earlier or no changes made, using the existing exclusion method !!!
    #datasets to remove (can be left empty if using the exclusion UI)
    frame_name = []

    all_correlations_mod = remove_dataframes(all_correlations, frame_name)
    
    #optional: if you want to show which files are being used
    print(f"Using {len(all_correlations_mod)} correlation datasets for analysis")

#update basedata to match the filtered correlation data
#get list of files to exclude (those not in the filtered correlations)
files_to_exclude = [f for f in df_basedata['filename'] 
                    if f not in all_correlations_mod.keys()]

#filter the basedata
df_basedata_mod = remove_from_data(df_basedata, files_to_exclude)

#re-index the basedata
df_basedata_mod = df_basedata_mod.reset_index(drop=True)  
df_basedata_mod.index = df_basedata_mod.index + 1

print(f"Base data filtered to {len(df_basedata_mod)} entries to match correlation data")

Using 39 correlation datasets for analysis
Base data filtered to 39 entries to match correlation data


In [16]:
#calculate mean and error for temperature and viscosity (for calculation of R_h)
mean_temperature = df_basedata_mod['temperature [K]'].mean()
std_temperature = df_basedata_mod['temperature [K]'].std()
sem_temperature = df_basedata_mod['temperature [K]'].sem()

mean_viscosity = df_basedata_mod['viscosity [cp]'].mean()
std_viscosity = df_basedata_mod['viscosity [cp]'].std()
sem_viscosity = df_basedata_mod['viscosity [cp]'].sem()

#std = standard deviation; sem = standard error
df_basedata_stats = pd.DataFrame({
    'mean temperature [K]': [mean_temperature], 
    'std temperature [K]': [std_temperature],
    'sem temperature [K]': [sem_temperature],
    'mean viscosity [cp]': [mean_viscosity],
    'std viscosity [cp]': [std_viscosity],
    'sem viscosity [cp]': [sem_viscosity],})
df_basedata_stats

,mean temperature [K],std temperature [K],sem temperature [K],mean viscosity [cp],std viscosity [cp],sem viscosity [cp]
0,297.940634,0.004605,0.000737,0.894452,0.000067,0.000011


In [17]:
#precalculate c and error for determination of Rh [ c = kb*T/6*pi*eta ] (using standard deviation)
from scipy.constants import k #import boltzmann-const.
c = (k*df_basedata_stats['mean temperature [K]'])/(6*np.pi*df_basedata_stats['mean viscosity [cp]']*10**(-3))

fractional_error_c = np.sqrt((df_basedata_stats['std temperature [K]'] / df_basedata_stats['mean temperature [K]'])**2 + (df_basedata_stats['std viscosity [cp]'] / df_basedata_stats['mean viscosity [cp]'])**2)
delta_c = fractional_error_c * c

print(f"c = {c.values[0]:.4e} +/- {delta_c.values[0]:.4e}")

relative_error_c = delta_c / c
print(f"Relative error in c: {relative_error_c.values[0]:.4%}")

c = 2.4398e-19 +/- 1.8548e-23
Relative error in c: 0.0076%


In [18]:
#correlation data from the software are normalized g(2)-1 data
#create a new dictionary with dataframes with time [s] and meanvalue of correlation 1 and 2 (has to be adjusted in preprocessing.py for other detectorsettings)
from preprocessing import process_correlation_data 
columns_to_drop = ['time [ms]', 'correlation 1', 'correlation 2', 'correlation 3', 'correlation 4']
processed_correlations_1 = process_correlation_data(all_correlations_mod, columns_to_drop)

In [19]:
perform_cumulant_A = False
if perform_cumulant_A:
    #extract cumulant data only for files in the filtered correlations
    from cumulants import extract_cumulants
    all_data = []
    #create mapping from filename to full path
    file_to_path = {os.path.basename(file): file for file in filtered_files}

    #only process files that are in the filtered correlations
    for filename in all_correlations_mod.keys():
        if filename in file_to_path:
            file_path = file_to_path[filename]
            extracted_cumulants = extract_cumulants(file_path)
            if extracted_cumulants is not None:
                extracted_cumulants['filename'] = filename  
                all_data.append(extracted_cumulants)
        else:
            print(f"Warning: No matching file found for {filename} in filtered_files")

    if all_data:
        df_extracted_cumulants = pd.concat(all_data, ignore_index=True)
        df_extracted_cumulants.index = df_extracted_cumulants.index + 1 
    else:
        print("No cumulant data extracted!")
        #create empty DataFrame to avoid errors in subsequent code
        df_extracted_cumulants = pd.DataFrame(columns=['filename', '1st order frequency [1/ms]', 
                                                 '2nd order frequency [1/ms]', '3rd order frequency [1/ms]',
                                                 '2nd order frequency exp param [ms^2]', 
                                                 '3rd order frequency exp param [ms^2]'])

    #if already defined files_to_exclude
    if 'files_to_exclude' in locals():
        df_extracted_cumulants_mod = remove_from_data(df_extracted_cumulants, files_to_exclude)
    else:
        #if required to re-filter because we're using all_correlations_mod
        df_extracted_cumulants_mod = df_extracted_cumulants

    #re-index
    df_extracted_cumulants_mod = df_extracted_cumulants_mod.reset_index(drop=True)  
    df_extracted_cumulants_mod.index = df_extracted_cumulants_mod.index + 1

    print(f"Extracted cumulant data for {len(df_extracted_cumulants_mod)} files")

    #create a combined dataframe of the modified dataframes (where some files may be removed)
    cumulant_method_A_data = pd.merge(df_basedata_mod, df_extracted_cumulants_mod, on = 'filename', how = 'outer')
    #re-index
    cumulant_method_A_data = cumulant_method_A_data.reset_index(drop=True)  
    cumulant_method_A_data.index = cumulant_method_A_data.index + 1
else:
    pass

In [20]:
#check data of cumulant method A
run_cell_cumulant_method_A_data = False

#
if run_cell_cumulant_method_A_data:
    print(cumulant_method_A_data)
else:
    print("Data check skipped.")

Data check skipped.


In [21]:
#plot cumulant data + linear regression to determine diffusion coefficient
if perform_cumulant_A:
    from cumulants import analyze_diffusion_coefficient
    cumulant_method_A_diff = analyze_diffusion_coefficient(
        data_df=cumulant_method_A_data, 
        q_squared_col='q^2', 
        gamma_cols=['1st order frequency [1/ms]', '2nd order frequency [1/ms]', '3rd order frequency [1/ms]'],
        gamma_unit='1/ms', #has to be 1/ms in case of the ALV data, default is 1/s
        #x_range=(0, 0.001) #ability to change the range of the fit
    )
else:
    pass

In [22]:
#export the plotted data as Cumulant_Method_A_data.txt
run_cell_cumulant_method_A_export = False

if run_cell_cumulant_method_A_export:
    cumulant_method_A_data.to_csv(f'Cumulant_Method_A_data_{experiment_name}.txt', sep='\t', index=False)
else:
    print("Plotted data for Cumulant Method A not exported.")

Plotted data for Cumulant Method A not exported.


In [23]:
#create a new dataframe with the diffusion-coefficient + error in m^2/s
if perform_cumulant_A:
    A_diff = pd.DataFrame()
    A_diff['D [m^2/s]'] = cumulant_method_A_diff['q^2_coef']*10**(-15)
    A_diff['std err D [m^2/s]'] = cumulant_method_A_diff['q^2_se']*10**(-15)
    print(A_diff)
else:
    pass

In [24]:
#calculate polydispersity-index
#not available for 1st order cumulant fit (for obvious reasons)
if perform_cumulant_A:
    cumulant_method_A_data['polydispersity_2nd_order'] = cumulant_method_A_data['2nd order frequency exp param [ms^2]']/(cumulant_method_A_data['2nd order frequency [1/ms]'])**2
    polydispersity_method_A_2 = cumulant_method_A_data['polydispersity_2nd_order'].mean()
    cumulant_method_A_data['polydispersity_3rd_order'] = cumulant_method_A_data['3rd order frequency exp param [ms^2]']/(cumulant_method_A_data['3rd order frequency [1/ms]'])**2
    polydispersity_method_A_3 = cumulant_method_A_data['polydispersity_3rd_order'].mean()
else:
    pass

In [25]:
#give results for cumulant method A
if perform_cumulant_A:
    from cumulants import calculate_cumulant_results_A
    method_A_cumulant_result = calculate_cumulant_results_A(A_diff, cumulant_method_A_diff, polydispersity_method_A_2, polydispersity_method_A_3, c, delta_c)
else:
    from cumulants import create_zero_cumulant_results_A
    method_A_cumulant_result = create_zero_cumulant_results_A()
method_A_cumulant_result

C:\Users\vince\Desktop\JADE-DLS-ReleaseVers\cumulants.py:350: SyntaxWarning: invalid escape sequence '\G'
  plt.ylabel(f'$\Gamma$ [{gamma_unit}]')
C:\Users\vince\Desktop\JADE-DLS-ReleaseVers\cumulants.py:352: SyntaxWarning: invalid escape sequence '\G'
  title = 'q$^2$ vs. $\Gamma$'


,Rh [nm],Rh error [nm],R_squared,Fit,Residuals,PDI
0,0,0,0,Rh from 1st order cumulant fit,0,NaN
1,0,0,0,Rh from 2nd order cumulant fit,0,0.0
2,0,0,0,Rh from 3rd order cumulant fit,0,0.0


In [26]:
perform_cumulant_B = False
#calculate sqrt(g2) for the dataframes (=g(2)_mod)
#checking if all g(2)-values are positive and dropping all that are below zero (negative values cannot be processed here)
#possibility to drop those below zero due to only requiring the data at very short times and negative values only appear near the baseline
if perform_cumulant_B:
    from cumulants import calculate_g2_B
    processed_correlations = calculate_g2_B(processed_correlations_1)
else:
    pass

In [27]:
#check the adapted correlation-data
run_cell_process_correlation = False

#
if run_cell_process_correlation:
    print(processed_correlations)
else:
    print("Data check skipped.")

Data check skipped.


In [28]:
#plot and fit the modified [g(2)-1]-data vs time
#keep fit-limits very narrow (depending on fit-results)
fit_limits = (0, 0.0002)

#fit function (up to 1st moment extension)
def fit_function(x, a, b, c):
    return 0.5*np.log(a) - b*x + 0.5*c*x**2
if perform_cumulant_B:
    from cumulants import plot_processed_correlations
    cumulant_method_B_fit = plot_processed_correlations(processed_correlations, fit_function, fit_limits)
else:
    pass

In [29]:
#create a combined dataframe with the basedata
if perform_cumulant_B:
    cumulant_method_B_data = pd.merge(df_basedata_mod, cumulant_method_B_fit, on = 'filename', how = 'outer')
    #re-index
    cumulant_method_B_data = cumulant_method_B_data.reset_index(drop=True)  
    cumulant_method_B_data.index = cumulant_method_B_data.index + 1 
else:
    pass

In [30]:
#check data of cumulant method B
run_cell_cumulant_method_B_data = False

#
if run_cell_cumulant_method_B_data:
    print(cumulant_method_B_data)
else:
    print("Data check skipped.")

Data check skipped.


In [31]:
#remove bad fits
if perform_cumulant_B:
    from cumulants import remove_rows_by_index
    indices_input = input("Enter row indices to remove (comma-separated): ")
    cumulant_method_B_data_mod = remove_rows_by_index(cumulant_method_B_data, indices_input)
else:
    pass

In [32]:
#plot and fit Gamma vs. q^2 for cumulant method B
if perform_cumulant_B:
    from cumulants import analyze_diffusion_coefficient
    cumulant_method_B_diff = analyze_diffusion_coefficient(
        data_df=cumulant_method_B_data_mod,
        q_squared_col='q^2',
        gamma_cols=['b'],
        method_names=['Method B'],
        #x_range=(0, 0.001) #change the range of the fit
)
else:
    pass

In [33]:
#export the plotted data as .txt
run_cell_cumulant_method_B_export = False

if run_cell_cumulant_method_B_export:
    cumulant_method_B_data.to_csv(f'Cumulant_Method_B_data_{experiment_name}.txt', sep='\t', index=False)  
else:
    print("Plotted data for Cumulant Method B not exported.")

Plotted data for Cumulant Method B not exported.


In [34]:
#create a new dataframe with the diffusion-coefficient + error in m^2/s
if perform_cumulant_B:
    B_diff = pd.DataFrame()
    B_diff['D [m^2/s]'] = cumulant_method_B_diff['q^2_coef']*10**(-18)
    B_diff['std err D [m^2/s]'] = cumulant_method_B_diff['q^2_se']*10**(-18)
    print(B_diff)
else:
    pass

In [35]:
#polydispersity index
if perform_cumulant_B:
    cumulant_method_B_data_mod['polydispersity'] = cumulant_method_B_data_mod['c']/(cumulant_method_B_data_mod['b'])**2
    polydispersity_method_B = cumulant_method_B_data_mod['polydispersity'].mean()
    print(polydispersity_method_B)
else:
    polydispersity_method_B = 0

In [36]:
#results for cumulant_method_B
if perform_cumulant_B:
    method_B_cumulant_result = pd.DataFrame()
    method_B_cumulant_result['Rh [nm]'] = c*(1/B_diff['D [m^2/s]'][0])*10**9
    fractional_error_Rh_B = np.sqrt((delta_c / c)**2 + (B_diff['std err D [m^2/s]'][0] / B_diff['D [m^2/s]'][0])**2)
    method_B_cumulant_result['Rh error [nm]'] = fractional_error_Rh_B * method_B_cumulant_result['Rh [nm]']
    method_B_cumulant_result['R_squared'] = cumulant_method_B_diff['R_squared']
    method_B_cumulant_result['Fit'] = 'Rh from linear cumulant fit'
    method_B_cumulant_result['Residuals'] = cumulant_method_B_diff['Normality']
    method_B_cumulant_result['PDI'] = polydispersity_method_B
else:
    method_B_cumulant_result = pd.DataFrame({
        'Rh [nm]': [0],
        'Rh error [nm]': [0],
        'R_squared': [0],
        'Fit': ['Rh from linear cumulant fit'],
        'Residuals': [0],
        'PDI': [polydispersity_method_B]
    })
    print("Cumulant-Method B not conducted.")
#
method_B_cumulant_result

Cumulant-Method B not conducted.


,Rh [nm],Rh error [nm],R_squared,Fit,Residuals,PDI
0,0,0,0,Rh from linear cumulant fit,0,0


In [37]:
perform_cumulant_C = False
if perform_cumulant_C:
    #fit-limits as broad as possible
    fit_limits = (1e-9, 10) #25ns to 10s should cover the whole range for our instrument

    #fit function (up to 4th cumulant)
    def fit_function4(x, a, b, c, d, e, f):
        inner_term = 1 + 0.5 * c * x**2 - (d * x**3) / 6 + ((e - 3 * c**2) * x**4) / 24
        term = f + a * (np.exp(-b * x) * inner_term)**2
        return term

    #fit function (up to 3rd cumulant) 
    def fit_function3(x, a, b, c, d, f):
        inner_term = 1 + 0.5 * c * x**2 - (d * x**3) / 6
        term = f + a * (np.exp(-b * x) * inner_term)**2
        return term
    
    #fit function (up to 2nd cumulant)
    def fit_function2(x, a, b, c, f):
        inner_term = 1 + 0.5 * c * x**2
        term = f + a * (np.exp(-b * x) * inner_term)**2
        return term
        
    #choose fit function
    chosen_fit_function = fit_function4
    
    #possible to choose a parameter adaption strategy and use the adapted parameters instead of the fixed base initial values
    adaptive_initial_guesses = True
    #if using adaptive parameters, choose adaption strategy:
    adaptation_strategy = 'individual'  #adapt initial values for each dataset individually
    #adaptation_strategy = 'global' #analyzes all datasets and uses median values for global parameters
    #adaptation_strategy = 'representative'  #picks the dataset with the best signal-to-noise ratio and uses its parameters for all datasets
    
    #define initial guesses
    initial_a = 0.8 #basically the starting point (usually around 0.8-0.9)
    initial_b = 10000 #basically decay-time -> large positive value in reasonable range
    initial_c = 0 #lower c = "smoother to baseline" (generelly c>0 for polydisperse samples)
    initial_d = 0 #higher d = "faster to baseline"
    initial_e = 0 #larger e = "later to baseline"
    initial_f = 0 #baseline is at about 0
    base_initial_parameters = [initial_a, initial_b, initial_c, initial_d, initial_e, initial_f] 

    if adaptive_initial_guesses:
        from cumulants_C import get_adaptive_initial_parameters
        #get adaptive parameters for all dataframes
        initial_parameters = get_adaptive_initial_parameters(
            processed_correlations_1,
            chosen_fit_function,
            base_initial_parameters,
            strategy=adaptation_strategy,
            verbose=True )
    else:
        from cumulants_C import get_meaningful_parameters
        #get only the meaningful parameters for this function
        initial_parameters = get_meaningful_parameters(chosen_fit_function, base_initial_parameters)

    #plotting and fitting
    from cumulants_C import plot_processed_correlations_iterative
    cumulant_method_C_fit = plot_processed_correlations_iterative(processed_correlations_1, chosen_fit_function, fit_limits, initial_parameters, method='lm')
    #3 optimization methods available: lm, trf or dogbox
else:
    pass

In [38]:
if perform_cumulant_C:
    #getting the fit-metric to compare multiple optimization algorithms
    from cumulants_C import calculate_mean_fit_metrics
    #get mean metrics across all datasets
    mean_metrics = calculate_mean_fit_metrics(cumulant_method_C_fit)
    #display results
    print(f"Mean R-squared: {mean_metrics['mean_R_squared']:.4f} ± {mean_metrics['std_R_squared']:.4f}")
    print(f"Mean RMSE: {mean_metrics['mean_RMSE']:.4e} ± {mean_metrics['std_RMSE']:.4e}")
    print(f"Mean AIC: {mean_metrics['mean_AIC']:.2f} ± {mean_metrics['std_AIC']:.2f}")
    print(f"Number of successfully fitted datasets: {mean_metrics['num_datasets']}")
else:
    pass

In [39]:
if perform_cumulant_C:
    #create a combined dataframe with the basedata
    cumulant_method_C_data = pd.merge(df_basedata_mod, cumulant_method_C_fit, on = 'filename', how = 'outer')
    #re-index
    cumulant_method_C_data = cumulant_method_C_data.reset_index(drop=True)  
    cumulant_method_C_data.index = cumulant_method_C_data.index + 1 
else:
    pass

In [40]:
#check data of cumulant method C
run_cell_cumulant_method_C_data = False

#
if run_cell_cumulant_method_C_data:
    print(cumulant_method_C_data)
else:
    print("Data check skipped.")

Data check skipped.


In [41]:
#remove bad fits
if perform_cumulant_C:
    from cumulants import remove_rows_by_index
    indices_input = input("Enter row indices to remove (comma-separated): ")
    cumulant_method_C_data_mod = remove_rows_by_index(cumulant_method_C_data, indices_input)
else:
    pass

In [42]:
#plot and fit Gamma vs. q^2 for cumulant method C
if perform_cumulant_C:
    from cumulants import analyze_diffusion_coefficient
    cumulant_method_C_diff = analyze_diffusion_coefficient(
        data_df=cumulant_method_C_data,
        q_squared_col='q^2',
        gamma_cols=['best_b'],
        method_names=['Method C'],
        #x_range=(0, 0.001) #change the range of the fit
    )
else:
    pass

In [43]:
#export the plotted data as .txt
run_cell_cumulant_method_C_export = False

if run_cell_cumulant_method_C_export:
    cumulant_method_C_data.to_csv(f'Cumulant_Method_C_data_{experiment_name}.txt', sep='\t', index=False)  
else:
    print("Plotted data for Cumulant Method C not exported.")

Plotted data for Cumulant Method C not exported.


In [44]:
#create a new dataframe with the diffusion-coefficient + error in m^2/s
if perform_cumulant_C:
    C_diff = pd.DataFrame()
    C_diff['D [m^2/s]'] = cumulant_method_C_diff['q^2_coef']*10**(-18)
    C_diff['std err D [m^2/s]'] = cumulant_method_C_diff['q^2_se']*10**(-18)
    print(C_diff)
else:
    pass

In [45]:
#polydispersity index
if perform_cumulant_C:
    cumulant_method_C_data['polydispersity'] = cumulant_method_C_data['best_c']/(cumulant_method_C_data['best_b'])**2
    polydispersity_method_C = cumulant_method_C_data['polydispersity'].mean()
    print(polydispersity_method_C)
else:
    polydispersity_method_C = 0

In [46]:
#results
if perform_cumulant_C:
    method_C_cumulant_result = pd.DataFrame()
    method_C_cumulant_result['Rh [nm]'] = c*(1/C_diff['D [m^2/s]'][0])*10**9
    fractional_error_Rh_C = np.sqrt((delta_c / c)**2 + (C_diff['std err D [m^2/s]'][0] / C_diff['D [m^2/s]'][0])**2)
    method_C_cumulant_result['Rh error [nm]'] = fractional_error_Rh_C * method_C_cumulant_result['Rh [nm]']
    method_C_cumulant_result['R_squared'] = cumulant_method_C_diff['R_squared']
    method_C_cumulant_result['Fit'] = 'Rh from iterative non-linear cumulant fit'
    method_C_cumulant_result['Residuals'] = cumulant_method_C_diff['Normality']
    method_C_cumulant_result['PDI'] = polydispersity_method_C
else:
    method_C_cumulant_result = pd.DataFrame({
        'Rh [nm]': [0],
        'Rh error [nm]': [0],
        'R_squared': [0],
        'Fit': ['Rh from iterative cumulant fit'],
        'Residuals': [0],
        'PDI': [polydispersity_method_C]
    })
    print("Cumulant-Method C not conducted.")
#   
method_C_cumulant_result

Cumulant-Method C not conducted.


,Rh [nm],Rh error [nm],R_squared,Fit,Residuals,PDI
0,0,0,0,Rh from iterative cumulant fit,0,0


In [47]:
#compare all results from cumulant fitting
df_all_cumulant_method_results = pd.concat([method_A_cumulant_result, method_B_cumulant_result, method_C_cumulant_result], ignore_index=True)
df_all_cumulant_method_results

,Rh [nm],Rh error [nm],R_squared,Fit,Residuals,PDI
0,0,0,0,Rh from 1st order cumulant fit,0,NaN
1,0,0,0,Rh from 2nd order cumulant fit,0,0.0
2,0,0,0,Rh from 3rd order cumulant fit,0,0.0
3,0,0,0,Rh from linear cumulant fit,0,0.0
4,0,0,0,Rh from iterative cumulant fit,0,0.0


In [48]:
#export cumulant-results-sheet as .txt
run_cell_cumulant_results_sheet = False

#
if run_cell_cumulant_results_sheet:
    cumulant_method_results.to_csv(f'Cumulant_Method_results_{experiment_name}.txt', sep='\t', index=False) 
else:
    print("Cumulant-Results-Sheet not exported.")

Cumulant-Results-Sheet not exported.


In [49]:
#simple NNLS-Fit without additional constraints
perform_nnls = False
#adjust parameters of NNLS-Fit
nnls_params = {
    'decay_times': np.logspace(-8, 1, 200), #define space and number of points in which the inverse laplacian problem is fitted
    'prominence': 0.05, #a measure how far from the baseline the peak has to be (lower means mmore sensitive peakpicking)
    'distance': 1} #minimum distance around the found peaks

from regularized import nnls, nnls_all
if perform_nnls:
    nnls_df = nnls_all(processed_correlations_1, nnls_params)
else:
    nnls_df = pd.DataFrame()

In [50]:
#create a combined dataframe with basedata and re-index
if perform_nnls:
    nnls_data = pd.merge(df_basedata_mod, nnls_df, on='filename', how='outer')
    nnls_data = nnls_data.reset_index(drop=True)
    nnls_data.index = nnls_data.index + 1
else:
    pass

In [51]:
#remove bad fits
from cumulants import remove_rows_by_index
if perform_nnls:
    indices_input = input("Enter row indices to remove (comma-separated): ")
    nnls_data_mod = remove_rows_by_index(nnls_data, indices_input)
else:
    pass

In [52]:
#calculate decay-rates
if perform_nnls:
    from regularized import calculate_decay_rates
    tau_cols = ['tau_1', 'tau_2', 'tau_3'] #can be extended for more peaks if required
    nnls_data_mod = calculate_decay_rates(nnls_data_mod, tau_cols)
else:
    pass

In [53]:
#plot data + linear regression to determine diffusion coefficient
if perform_nnls:
    from cumulants import analyze_diffusion_coefficient
    nnls_diff = analyze_diffusion_coefficient(
        data_df=nnls_data_mod,
        q_squared_col='q^2',
        gamma_cols=['gamma_1', 'gamma_2', 'gamma_3'], #can be extended for more than 3 peaks again
        #x_range=(0, 0.001) #change the range of the fit
    ) 
else:
    pass

In [54]:
#export the plotted data as .txt
run_cell_nnls_export = False

#
if run_cell_nnls_export:
    nnls_data_mod.to_csv(f'nnls_results_{experiment_name}.txt', sep='\t', index=False) 
else:
    print("Plotted data for NNLS regression not exported.")

Plotted data for NNLS regression not exported.


In [55]:
#create a new dataframe with the diffusion-coefficient + error in m^2/s
if perform_nnls:
    nnls_diff_results = pd.DataFrame()
    nnls_diff_results['D [m^2/s]'] = nnls_diff['q^2_coef']*10**(-18)
    nnls_diff_results['std err D [m^2/s]'] = nnls_diff['q^2_se']*10**(-18)
    print(nnls_diff_results)
else:
    pass

In [56]:
#iterate through the multiple fits
from IPython.display import display
try:
    temp_results = []
    for i in range(len(nnls_diff_results)):
        result = pd.DataFrame()
        result['Rh [nm]'] = c * (1 / nnls_diff_results['D [m^2/s]'][i]) * 10**9
        fractional_error_Rh = np.sqrt((delta_c / c)**2 + (nnls_diff_results['std err D [m^2/s]'][i] / nnls_diff_results['D [m^2/s]'][i])**2)
        result['Rh error [nm]'] = fractional_error_Rh * result['Rh [nm]']
        result['R_squared'] = [nnls_diff['R_squared'][i]]
        result['Fit'] = [f'Rh from fit tau_{i+1}']
        result['Residuals'] = [nnls_diff['Normality'][i]]
        temp_results.append(result)
    df_final_nnls_results = pd.concat(temp_results, ignore_index=True)
    display(df_final_nnls_results)
except NameError:
    print("NNLS-Fit not performed. Process cancelled.")

NNLS-Fit not performed. Process cancelled.


In [57]:
#predetermine a suitable alpha-value by analyzing a random set of data with different alpha-values
analyze_alpha = False
from regularized import nnls_reg_simple, analyze_random_datasets_grid
if analyze_alpha:
    base_params = {
    'decay_times': np.logspace(-8, 1, 200), #define space and numer of points in which the inverse laplacian problem is fitted
    'prominence': 0.01, #a measure how far from the baseline the peak has to be (lower means mmore sensitive peakpicking)
    'distance': 1} #minimum distance around the found peaks

    fig, selected_datasets = analyze_random_datasets_grid(
    processed_correlations_1, 
    num_datasets=3, #number of randomly selected datasets
    base_nnls_params=base_params, 
    nnls_reg_simple_function=nnls_reg_simple, #also possible with nnls_reg
    alpha_range=(0.1, 1), #set alpha range (logarithmic spacing between values)
    num_alphas=5) #number of alphas in range 
else:
    print("alpha-analysis skipped.")

alpha-analysis skipped.


In [58]:
#regularized fit (NNLS-fit with Tikhonov-Phillips regulation)
regularized_fit = False
#adjust parameters of NNLS-Reg-Fit
nnls_reg_params = {
    'decay_times': np.logspace(-8, 1, 200), #define space and number of points in which the inverse laplacian problem is fitted
    'prominence': 0.01, #a measure how far from the baseline the peak has to be (lower means mmore sensitive peakpicking)
    'distance': 1, #minimum distance around the found peaks
    'alpha': 0.5, #alpha-value
    'normalize': True, #enable for normalization           
    'sparsity_penalty': 0, #>0 introduces a sparsity penalty
    'enforce_unimodality': False #if true, enforces the result to be one peak, can be used for monomodal samples as comparison to cumulant analysis
}
if regularized_fit:
    from regularized import nnls_reg, nnls_reg_all
    nnls_reg_df, full_results = nnls_reg_all(processed_correlations_1, nnls_reg_params)
else:
    pass

In [59]:
#create a combined dataframe with basedata and re-index
if regularized_fit:
    nnls_reg_data = pd.merge(df_basedata_mod, nnls_reg_df, on='filename', how='outer')
    nnls_reg_data = nnls_reg_data.reset_index(drop=True)
    nnls_reg_data.index = nnls_reg_data.index + 1
else:
    pass

In [60]:
#remove bad fits
if regularized_fit:
    from cumulants import remove_rows_by_index
    indices_input = input("Enter row indices to remove (comma-separated): ")
    nnls_reg_data_mod = remove_rows_by_index(nnls_reg_data, indices_input)
else:
    pass

In [61]:
#calculate decay-rates
if regularized_fit:
    from regularized import calculate_decay_rates
    tau_cols = ['tau_1', 'tau_2', 'tau_3', 'tau_4'] #can be extended for more peaks if required
    nnls_reg_data_mod = calculate_decay_rates(nnls_reg_data_mod, tau_cols)
else:
    pass

In [62]:
#plot data + linear regression to determine diffusion coefficient
if regularized_fit:
    from cumulants import analyze_diffusion_coefficient
    nnls_reg_diff = analyze_diffusion_coefficient(
        data_df=nnls_reg_data_mod,
        q_squared_col='q^2',
        gamma_cols=['gamma_1', 'gamma_2', 'gamma_3'], #can be extended for more peaks again
        #x_range=(0, 0.001) #change the range of the fit
    )
else:
    pass

In [63]:
#export the plotted data as .txt
run_cell_nnls_reg_export = False

#
if run_cell_nnls_reg_export:
    nnls_reg_data_mod.to_csv(f'nnls_reg_results_{experiment_name}.txt', sep='\t', index=False) 
else:
    print("Plotted data for regularized NNLS regression not exported.")

Plotted data for regularized NNLS regression not exported.


In [64]:
#create a new dataframe with the diffusion-coefficient + error in m^2/s
if regularized_fit:
    nnls_reg_diff_results = pd.DataFrame()
    nnls_reg_diff_results['D [m^2/s]'] = nnls_reg_diff['q^2_coef']*10**(-18)
    nnls_reg_diff_results['std err D [m^2/s]'] = nnls_reg_diff['q^2_se']*10**(-18)
    print(nnls_reg_diff_results)
else:
    pass

In [65]:
#iterate through the multiple fits
from IPython.display import display
try:
    temp_results = []
    for i in range(len(nnls_reg_diff_results)):
        result = pd.DataFrame()
        result['Rh [nm]'] = c * (1 / nnls_reg_diff_results['D [m^2/s]'][i]) * 10**9
        fractional_error_Rh = np.sqrt((delta_c / c)**2 + (nnls_reg_diff_results['std err D [m^2/s]'][i] / nnls_reg_diff_results['D [m^2/s]'][i])**2)
        result['Rh error [nm]'] = fractional_error_Rh * result['Rh [nm]']
        result['R_squared'] = [nnls_reg_diff['R_squared'][i]]
        result['Fit'] = [f'Rh from fit tau_{i+1}']
        result['Residuals'] = [nnls_reg_diff['Normality'][i]]
        temp_results.append(result)
    df_final_nnls_reg_results = pd.concat(temp_results, ignore_index=True)
    display(df_final_nnls_reg_results)
except NameError:
    print("Regularized Fit not performed. Process cancelled.")

Regularized Fit not performed. Process cancelled.


In [67]:
#compare different angles of the regularized fit
compare_results_reg = False

if compare_results_reg:
    from regularized import plot_distributions
    plot_distributions(full_results, nnls_reg_params, nnls_reg_data, 
                   angles=[30,90,150], #choose angles to plot
                   measurement_mode='average', #average: plots the average of that angle, first: plots first distribution of that angle, all: plots all distributions 
                   convert_to_radius=True, #plot Rh instead of decay time
                   figsize=(8, 6), #figure size
                   title="Distribution Comparison" #title
                  )
else:
    pass